# TRANSFORM
# Cleaning raw data from Amazon.es

Amazon data was scraped on 2023-10-04 using `selenium` python module. <br/> 
Now the data must be cleaned and transformed to comply with the DDBB standards.

The raw datasets have the following **structure**: <br/>

* Dictionary of lists - Each key divides the clothes by colors <br/>
* Color list of lists - Each list divides the clothes by amazon search page <br/>
* Each element of the list collects the data relative to one product <br/>

The raw datasets have the following **issues to amend**:

1. Normalize the colors names using `mlg.namvector_clean` function <br/>
2. Keep only **brand, product description and price €**  <br/>
     Functions to filter data: `prod_clean_amz1`, `complete_price_amz1` and `relev_inf_amz1` <br/>
3. Clean `price` column. Clean this kind of cells: 
    * 30.00 €(1 nueva oferta)
    * 24.29,99€ --- > 29.99€
    * Take out € sign and transform to float <br/>
    <br/>
4. Drop rows with wrong `price/description` and/or `brand/description`


### 0. Import modules

In [9]:
import pickle
import pandas as pd
import warnings

#import pylab as plt   # import matplotlib.pyplot as plt#
import matplotlib.pyplot as plt

# print the plot in the jupyter output
%matplotlib inline 

from src import ETL_functions as etl
from src import dataanalysis_fun1 as eda

Reload my module if neccessary

In [ ]:
import importlib
from src import dataanalysis_fun1 as eda # Import the module
#importlib.reload(mlg)  # Reload the module

# Suppress warning when reloading the module
with warnings.catch_warnings():
    warnings.simplefilter("ignore") 
    importlib.reload(mlg)  # Reload the module

## 1. Load the data using pickle

In [8]:
with open('../data/raw/amz_men_tshirt_data.pkl', 'rb') as file:
    amz_men_tshirt_data = pickle.load(file)
    
with open('../data/raw/amz1_womentshirt_data.pkl', 'rb') as file:
    amz1_womentshirt_data = pickle.load(file)

## 2. Transform/clean the data 

First for women clothes, then for men dataset

### 2.1 Normalize the key names (colors)

In [12]:
### fisrt for women
amz1_raw=amz1_womentshirt_data

## 
keys_list=amz1_raw.keys() # get the original keys
keys_list2=eda.namvector_clean(keys_list) # use the edited keys
key_changes = dict(zip(keys_list, keys_list2))
amz1_raw2 = {key_changes.get(key, key): value for key, value in amz1_raw.items()} # change the keys

dict_keys(['negro', 'gris', 'blanco', 'marron', 'beis', 'rojo', 'rosa', 'naranja', 'amarillo', 'marfil', 'verde', 'turquesa', 'azul', 'morado', 'dorado'])

### 2.2 Filter out irrelevant data

Run the cleaning functions -available in `src/ETL_functions.py`- along pages and colors

In [13]:
color_list=list(amz1_raw2.keys())

keys_discard=["+", "Entrega", "Prime", "PVPR", "Patrocinado", "stock", "PROMO","Personalizarlo",
              "Oferta", "Ahorra", "Envío", "opción", "vendido", "Antes"]

color_df=pd.DataFrame(columns=["brand", "description", "price", "colour"])

for j in range(len(color_list)):
    test_color=amz1_raw2[color_list[j]]
    test_color1=[etl.prod_clean_amz1(test,keys_discard) for test in test_color]
    test_color2=[etl.complete_price_amz1(test) for test in test_color1]
    test_color3=[etl.relev_inf_amz1(test, color_list[j]) for test in test_color2]

    for pag in range(len(test_color3)): #by pages
        # print(color_list[j], "page:", pag)   
        
        # check the content of elements with size not equal 4
        ## [print((test)) for test in test_color3[pag] if len(test)!=4]

        test_color4=[test for test in test_color3[pag] if len(test)==4]

        for row in test_color4:
            row_df = pd.DataFrame([row], columns=color_df.columns)
            color_df = pd.concat([color_df, row_df], ignore_index=True)


### 2.3 Clean `price` column


In [30]:
color_df1=color_df.copy()
color_df1["price"]=[VAL.split("(")[0] for VAL in color_df1["price"]]
color_df1["price"]=[VAL.replace(" ", "") for VAL in color_df1["price"]]
color_df1["price"]=[VAL.replace("€", "") for VAL in color_df1["price"]]

In [31]:
for i in range(len(color_df1["price"])):
    if len(color_df1["price"][i].split(".")) > 3:
        #print(color_df1["price"][i].split("."))  # Drop rows with price > 3 elements
        color_df1['price'][i] = ""
    elif len(color_df1["price"][i].split(",")) == 2:
        try:
            #print(color_df1['price'][i].split(".")[-1])
            color_df1['price'][i] = color_df1['price'][i].split(".")[-1]
        except:
            row_todel+=i

color_df1 = color_df1[color_df1.price != ""]

### 2.4 Drop rows with wrong `price/description`

In [32]:
color_df1=color_df1[color_df1["price"].apply(len)<10]
color_df1 = color_df1[color_df1["price"] != ""]
color_df1["price"]= color_df1['price'].str.replace(',', '.', regex=True)
color_df1['price'] = color_df1['price'].astype(float)
color_df1=color_df1[color_df1["brand"].apply(len)<30]

## 3. Save cleaned data

Comment lines to avoid overwriting

In [34]:
#amz1_women_clothes_clean=color_df1.copy()
#amz1_women_clothes_clean.to_csv('../data/clean/amz1_women_clothes_clean.csv', index=False)

## Men clothes dataset

In [35]:
### for men
amz1_raw=amz_men_tshirt_data

## 
keys_list=amz1_raw.keys() # get the original keys
keys_list2=eda.namvector_clean(keys_list) # use the edited keys
key_changes = dict(zip(keys_list, keys_list2))
amz1_raw2 = {key_changes.get(key, key): value for key, value in amz1_raw.items()} # change the keys

In [36]:
color_list=list(amz1_raw2.keys())

keys_discard=["+", "Entrega", "Prime", "PVPR", "Patrocinado", "stock", "PROMO","Personalizarlo",
              "Oferta", "Ahorra", "Envío", "opción", "vendido", "Antes"]

color_df=pd.DataFrame(columns=["brand", "description", "price", "colour"])

for j in range(len(color_list)):
    test_color=amz1_raw2[color_list[j]]
    test_color1=[etl.prod_clean_amz1(test,keys_discard) for test in test_color]
    test_color2=[etl.complete_price_amz1(test) for test in test_color1]
    test_color3=[etl.relev_inf_amz1(test, color_list[j]) for test in test_color2]

    for pag in range(len(test_color3)): #by pages
        # print(color_list[j], "page:", pag)   
        
        # check the content of elements with size not equal 4
        ## [print((test)) for test in test_color3[pag] if len(test)!=4]

        test_color4=[test for test in test_color3[pag] if len(test)==4]

        for row in test_color4:
            row_df = pd.DataFrame([row], columns=color_df.columns)
            color_df = pd.concat([color_df, row_df], ignore_index=True)


In [37]:
color_df1=color_df.copy()
color_df1["price"]=[VAL.split("(")[0] for VAL in color_df1["price"]]
color_df1["price"]=[VAL.replace(" ", "") for VAL in color_df1["price"]]
color_df1["price"]=[VAL.replace("€", "") for VAL in color_df1["price"]]

In [38]:
for i in range(len(color_df1["price"])):
    if len(color_df1["price"][i].split(".")) > 3:
        #print(color_df1["price"][i].split("."))  # Drop rows with price > 3 elements
        color_df1['price'][i] = ""
    elif len(color_df1["price"][i].split(",")) == 2:
        try:
            #print(color_df1['price'][i].split(".")[-1])
            color_df1['price'][i] = color_df1['price'][i].split(".")[-1]
        except:
            row_todel+=i

color_df1 = color_df1[color_df1.price != ""]

In [39]:
color_df1=color_df1[color_df1["price"].apply(len)<10]
color_df1 = color_df1[color_df1["price"] != ""]
color_df1["price"]= color_df1['price'].str.replace(',', '.', regex=True)
color_df1['price'] = color_df1['price'].astype(float)
color_df1=color_df1[color_df1["brand"].apply(len)<30]

In [41]:
#amz1_men_clothes_clean=color_df1.copy()
#amz1_men_clothes_clean.to_csv('../data/clean/amz1_men_clothes_clean.csv', index=False)

In [44]:
display(amz1_women_clothes_clean.head()), display(amz1_men_clothes_clean.head())

,brand,description,price,colour
0,Ranphee,Camisetas Túnica de Manga Larga para Mujer Blu...,25.99,negro
1,ANGGREK,Mujer Verano Tops Casual Encaje Cuello en V Ca...,31.19,negro
2,GRMLRPT,2023 Camiseta de Manga 3/4 Mujer Moda Cuello e...,25.99,negro
3,Urban Classics,Ladies Extended Shoulder tee Camiseta para Mujer,11.90,negro
4,Only,"Printed 3/4 Sleeved Top Suéter, Nero (Black/AO...",19.99,negro


,brand,description,price,colour
0,Jack & Jones,Hombres Sudaderas jjeCorp Logo,34.99,negro
1,Mister Tee,"sudadera con capucha, manga larga, de hombre, ...",39.90,negro
2,Joma,"Academy IV Sudadera con Capucha, Hombre",26.50,negro
3,Geographical Norway,Gymclass Sudadera para Hombre,39.90,negro
4,Joma,Sudadera Faraon para Hombre,21.50,negro


(None, None)